In [0]:


import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)
print(PLPROXY_DSN)

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list


begin_date = datetime(2020, 7, 1)
end_date = datetime(2020, 8, 31)

DATE_GRANULARITY_MAPPINGLIST = {
    "daily": get_date_list(begin_date, end_date, "D"),
    "weekly": get_date_list(begin_date, end_date, "W-SAT"),
    "monthly": get_date_list(begin_date, end_date, "M")
}


DATE_GRANULARITY_MAPPINGLIST["monthly"]
DATE_GRANULARITY_MAPPINGLIST["weekly"]
DATE_GRANULARITY_MAPPINGLIST["daily"]

print DATE_GRANULARITY_MAPPINGLIST["monthly"]
print DATE_GRANULARITY_MAPPINGLIST["weekly"]
print DATE_GRANULARITY_MAPPINGLIST["daily"]

In [0]:

import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY

app_basic_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from mu.app_{granularity}
    where date='{date}'
        and kpi=1
        and estimate is not null
        and estimate > 0
$proxy$) tpl (count bigint);"""


segment_by_age_gender_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from ag.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""

segment_by_audience_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from ad.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""
# check the country
# segment_by_product_sql = """
# select distinct store_id AS store_id from plproxy.execute_select_nestloop($proxy$ 
#     select distinct store_id AS store_id
#     from au.app_monthly
#     WHERE date='2015-01-31'
#         and kpi=1
#         and estimate is not null
#         and estimate <> 0

# $proxy$) tpl (store_id int) order by store_id;
# """

segment_by_product_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from au.app_{granularity}
    WHERE date='{date}'
    and kpi=1
    and estimate is not null
    and estimate <> 0

$proxy$) tpl (count bigint);
"""

app_cross_app = """
select distinct store_id AS store_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct store_id AS store_id
    from ca.app_monthly
    where 
        date ='2015-01-31'
        and kpi = 9
$proxy$) tpl (store_id int) order by store_id;
"""

# app_retention = """
# select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
#     select count(1) as count
#     from rt.app_{granularity}
#     where 
#         date ='{date}'
# $proxy$) tpl (count bigint) ;
# """

app_retention = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from rt.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""

granularity_list = ["monthly"]

def get_plproxy_data(sql_str):
    for granularity in granularity_list:
        for date_str in DATE_GRANULARITY_MAPPINGLIST[granularity]:
            result = query(PLPROXY_DSN, sql_str.format(granularity=granularity,date=date_str))
            count = result[0][0]
            print str(granularity)+str(" ")+str(date_str)+str(" ")+str(count)

# def get_plproxy_data(sql_str):
#     result = query(PLPROXY_DSN, sql_str)
#     count = result[0][0]
#     for r in result:
#         print(r[0])
#     # print str("monthly")+str(" ")+str("2017-05-31")+str(" ")+str(result)
            
def get_unified_data():
    print 1

get_plproxy_data(segment_by_audience_sql)  

In [0]:

basic_dump="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-mu_app.v2/fact/granularity=monthly/"
basic="s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/"
segment_by_age_gender_sql="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ag_app.v2/fact/granularity=/"
segment_by_audience="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ad_app.v2/fact/granularity=monthly/"
segment_by_product_sql="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity=weekly/month=2015-01"
app_cross_app="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ca_app.v2/fact/granularity=monthly/"
app_retention="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-rt_app.v2/fact/granularity=monthly/"
start_str="2015-01"
end_str="2017-05"

# segment_by_product_sql=spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact").parquet(segment_by_product_sql)
app_cross_app=spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ca_app.v2/fact").parquet(app_cross_app).where("month>='{}'".format(start_str)).where("month<='{}'".format(end_str))
# basic = spark.read.format('delta').load(basic).where("date>='{}'".format(start_str)).where("date<='{}'".format(end_str))
# segment_by_product_sql = spark.read.format('delta').load(segment_by_product_sql)
# print("count:"+str(basic_dump.count().show()))
app_cross_app.createOrReplaceTempView("app_cross_app")
# print("segment_by_product_sql.count(): "+str(segment_by_product_sql.filter("kpi=1").groupBy("date").count().orderBy("date").show(1500,truncate=False)))

app_cross_app= spark.sql("""
                SELECT date,count(1)
                FROM app_cross_app
                WHERE kpi=9
                and estimate is not null
                and estimate <> 0
                group by date
                order by date
                """)
basic= spark.sql("""
                SELECT date,count(1)
                FROM basic
                WHERE est_average_active_users is not null
                and est_average_active_users <> 0
                group by date
                order by date
                """)
segment_by_product_sql= spark.sql("""
                SELECT distinct store_id,date
                FROM segment_by_product_sql
                WHERE kpi=1
                and estimate is not null
                and estimate <> 0
                group by date,store_id
                order by date,store_id
                """)
app_cross_app= spark.sql("""
                SELECT date,count(1)
                FROM app_cross_app
                WHERE kpi=9
                and estimate is not null
                and estimate <> 0
                group by date
                order by date
                """)
app_cross_app.show(1000,False)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-ad_app.v2/fact/granularity=monthly/


In [0]:

import pandas as pd
from pyspark.sql import functions as F
from datetime import datetime
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType
from applications.db_check_v1.common.db_check_utils import query
from conf.settings import PG_USAGE_HOSTS, PG_USAGE_NAME, PG_USAGE_ACCESS_ID, PG_USAGE_SECRET_KEY, \
    CITUS_USAGE_NAME, CITUS_USAGE_ACCESS_ID, CITUS_USAGE_HOSTS, CITUS_USAGE_SECRET_KEY


PLPROXY_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_USAGE_NAME,
        user=PG_USAGE_ACCESS_ID,
        host=PG_USAGE_HOSTS[0][0],
        password=PG_USAGE_SECRET_KEY,
        port=PG_USAGE_HOSTS[0][1]
    )
)

app_basic_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from mu.app_{granularity}
    where date='{date}'
        and kpi=1
        and estimate is not null
        and estimate > 0
$proxy$) tpl (count bigint);"""


segment_by_age_gender_sql = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from ag.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""
# check the country
segment_by_product_sql = """
select distinct store_id AS store_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct store_id AS store_id
    from au.app_weekly
    WHERE date='2015-01-03'
        and kpi=1
        and estimate is not null
        and estimate <> 0

$proxy$) tpl (store_id int) order by store_id;
"""

# segment_by_product_sql = """
# select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
#     select count(1) as count
#     from au.app_{granularity}
#     WHERE date='{date}'
#     and kpi=1
#     and estimate is not null
#     and estimate <> 0

# $proxy$) tpl (count bigint);
# """

app_cross_app = """
select distinct store_id AS store_id from plproxy.execute_select_nestloop($proxy$ 
    select distinct store_id AS store_id
    from ca.app_monthly
    where 
        date ='2015-07-31'
        and kpi = 9
$proxy$) tpl (store_id int) order by store_id;
"""

# app_retention = """
# select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
#     select count(1) as count
#     from rt.app_{granularity}
#     where 
#         date ='{date}'
# $proxy$) tpl (count bigint) ;
# """

app_retention = """
select sum(count) as count from plproxy.execute_select_nestloop($proxy$ 
    select count(1) as count
    from rt.app_{granularity}
    where 
        date ='{date}'
$proxy$) tpl (count bigint) ;
"""

granularity_list = ["monthly","weekly"]

# def get_plproxy_data(sql_str):
#     for granularity in granularity_list:
#         for date_str in DATE_GRANULARITY_MAPPINGLIST[granularity]:
#             result = query(PLPROXY_DSN, sql_str.format(granularity=granularity,date=date_str))
#             count = result[0][0]
#             print str(granularity)+str(" ")+str(date_str)+str(" ")+str(count)

def get_plproxy_data(sql_str):
    plproxy_result = []
    result = query(PLPROXY_DSN, segment_by_product_sql)
    df_data = [Row(store_id=r[0]) for r in result]
    _schema =StructType([StructField("store_id", IntegerType(), False)])
    df_plproxy = spark.createDataFrame(data=df_data, schema=_schema)
    return df_plproxy
            
def get_unified_data():
    segment_by_product_sql1="s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact/granularity=weekly/month=2015-01"
    segment_by_product_sql1=spark.read.option("basePath", "s3://b2c-prod-data-pipeline-unified-usage/unified/app-tech.usage.legacy-au_app.v2/fact").parquet(segment_by_product_sql1)
    # spark.read.format("delta").load(unified_source_path).createOrReplaceTempView("test_unified")
    segment_by_product_sql1.createOrReplaceTempView("segment_by_product_sql1")
    segment_by_product_sql1= spark.sql("""
                SELECT store_id
                FROM segment_by_product_sql1
                WHERE kpi=1
                and estimate is not null
                and estimate <> 0
                group by store_id
                order by store_id
                """)
    segment_by_product_sql1.createOrReplaceTempView("segment_by_product_sql1")

def compare():
    df_plproxy=get_plproxy_data(segment_by_product_sql)
    get_unified_data()
    df_plproxy.createOrReplaceTempView("plproxy_df_new")
    spark.sql("""
                select distinct store_id 
                from segment_by_product_sql1
                 except 
                    select distinct store_id 
                    from plproxy_df_new 
                    order by store_id
                    """).show(100,False)


# df = get_plproxy_data(segment_by_product_sql)
compare()


